# Changes in recent versions


The underlying raw converted data structure used by echopype (the [`EchoData`](data-format:echodata-object) object) has undergone revisions over time. While in most echopype releases these changes are relatively small, versions 0.8.0 and 0.6.0 incorporated significant changes, with implications for backward compatibility. Here we describe the main changes in each of these major releases. Please refer to [What's new](whats-new.md) for more complete details on specific changes that impacted the data structure.

(data-format:v0.7.1-to-v0.8.0-changes)=
## v0.7.1 to v0.8.0 changes

Changes introduced in version 0.8.0 were carried out to incorporate missing variables that are mandated by SONAR-netCDF4 v1 and to implement adaptations to the convention in a more consistent fashion across variables and instrument types. Some of these changes modified or reverted decisions implemented in version 0.6.0 (below) that were later found to have large impacts on performance and usability.

Highlights include:
- Removed `beam` and `ping_time` dimensions from a whole netCDF4 group or individual variables when they were determined not to be required for specific instrument types. The `beam` dimension was dropped from all `Sonar/Beam_groupX` groups except for EK80 complex samples, where both `backscatter_r` and `backscatter_i` exist and the `beam` dimension represents different sectors of split-beam transducers and is required for these two variables. The `ping_time` dimension was retained only with variables that are known to potentially vary with time in the instrument types supported by echopype.
- Standardized the use of `transmit_frequency_start` and `transmit_frequency_stop` in `Sonar/Beam_groupX` groups, where they were previously missing or the names being used (`frequency_start` and `frequency_end`) were not the ones specified by the convention.
- In the `Platform` group: implemented the occurrence of convention-based variables more consistently across instrument types, including their default values when no values were found in the raw data file; and dimensions assigned to each variable were revised and assigned consistently across instrument types, with dimensions deemed unnecessary dropped from some variables.
- Added new `Provenance` group `combination_*` attributes to the "combined" `EchoData` object, mirroring the convention-based `conversion_*` attributes.
- In the `Vendor_specific` group, moved filter coefficients and decimation factor from attributes to variables in EK80, to facilitate consistent provenance tracking during `combine_echodata` operations.
- Improved the presence and use of variable attributes throughout `EchoData` groups.

Version 0.8.0 does not incorporte the capability to read files converted by previous versions of echopype. We recommend using `open_raw` to re-convert the raw data files.

(data-format:v0.5.x-to-v0.6.0-changes)=
## v0.5.x to v0.6.0 changes

In order to enhance the compliance of echopype-generated datasets to the SONAR-netCDF4 version 1 convention, a number of changes were introduced in echopype v0.6.0 that create incompatibilities with the data structure used in previous versions.

To ease the transition, the [`open_converted`](function:open-converted) function is able to open files previously converted using echopype v0.5.x (0.5.0 to 0.5.6) into the v0.6.0 data format, encapsulated in the `EchoData` object.

Key changes involved renaming and restructuring a couple of groups, and renaming some coordinates and data variables, as summarized below:

| Type       | v0.5.x          | v0.6.0                 | Rationale and notes      |
| :--------- | :-------------- | :--------------------- | :----------------------- |
| Group      | `Beam`          | `Sonar/Beam_group1`    | Convention compliance |
| Group      | `Beam_power`    | `Sonar/Beam_group2`    | Convention compliance |
| Group      | `Vendor`        | `Vendor_specific`      | Convention compliance |
| Coordinate | `frequency`     | `channel`              | Accommodate channels with duplicated frequencies. The new variable `frequency_nominal` was introduced |
| Coordinate | `range_bin`     | `range_sample`         | Better intuitive understanding of data |
| Coordinate | `quadrant`      | `beam`                 | Convention compliance. This `Beam_groupX` coordinate was added when it did not exist. |
| Coordinate | `location_time` | `time1`                | Convention compliance. In `Platform` group |
| Coordinate | `mru_time`      | `time2`                | Convention compliance. In `Platform` group |
| Variable   | `heave`         | `vertical_offset`      | Convention compliance. In `Platform` group |
| Variable   | `src_filenames` | `source_filenames`     | Convention compliance. In `Provenance` group. Also converted from global attribute to variable |

Other changes included:
- Adding previously missing, mandatory convention variables. When no data are available to populate them, these are filled with null (`NaN`) values.
- Moving variables from one group to another, particularly from the Beam groups to `Platform` and `Vendor`. These variables were not typically not part of the convention.
- The Beam_groupX `beamwidth_receive_athwartship` and `beamwidth_transmit_athwartship` variables were consolidated into `beamwidth_twoway_athwartship` because the EK60 and EK80 echosounders do not store one-way transmit or receive beam widths. Likewise for `beamwidth_receive_alongship` and `beamwidth_transmit_alongship`.

More details, including Pull Requests and discussions related to these changes, can be found in the [Release notes](whats-new.html#v0-6-0-2022-may-26).

### Convert old files to v0.6.0 format

To convert data files from v0.5.x to v0.6.0 format, simply open the old files and re-save them:

```python
import echopype as ep
# open old v0.5.x file and convert it into a v0.6.0-format EchoData object
ed = ep.open_converted("old_format_file.nc")
ed.to_netcdf("new_format_file.nc")
```

### v0.5.x data format

Below we provide a sample of the v0.5.x data format via a printout of the previous `EchoData` object.

Compare this with the [v0.6.0 `EchoData` object](data-format:echodata-object) to see the changes listed in the table above.

In [2]:
ed

EchoData: standardized raw data from Internal Memory
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.